# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [1]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [40]:
# 1) Seu código aqui
url = 'http://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Estado'
r = requests.get(url)
data_json = r.json()
df = pd.DataFrame(data_json['items'])
df.head()

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,13560,Estado,Rio de Janeiro,33,RJ,None,Deferido,19405.000026/2002-01,2002-06-25T03:00:00Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,49942.0,1,0,27/06/2002
1,5918,Estado,Rio de Janeiro,33,RJ,None,Encaminhado à PGFN com manifestação técnica fa...,17944.000233/2012-05,2012-04-16T03:00:00Z,Operação contratual externa (com garantia da U...,Infraestrutura,Instituição Financeira Internacional,Corporação Andina de Fomento,Dólar dos EUA,319675000.0,1,0,11/03/2016
2,13624,Estado,Rio de Janeiro,33,RJ,None,Deferido,19405.000039/2002-72,2002-06-20T03:00:00Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,67482400.0,1,0,27/06/2002
3,10562,Estado,Rio de Janeiro,33,RJ,None,Encaminhado à PGFN com manifestação técnica fa...,17944.001228/2013-92,2013-10-22T02:00:00Z,Operação contratual externa (com garantia da U...,Infraestrutura,Instituição Financeira Internacional,Banco Internacional para Reconstrução e Desenv...,Dólar dos EUA,500000000.0,1,0,27/12/2013
4,12375,Estado,Rio de Janeiro,33,RJ,None,Encaminhado à PGFN com manifestação técnica fa...,17944.001671/2011-00,2011-10-18T02:00:00Z,Operação contratual externa (com garantia da U...,Preservação e recuperação ambiental,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,451980000.0,1,0,27/12/2011


In [12]:
# 2) Seu código aqui
status = df.status.value_counts()
status.head(3)

Encaminhado à PGFN com manifestação técnica favorável    34
Deferido                                                 14
Arquivado a pedido                                        3
Name: status, dtype: int64

In [24]:
# 3) Seu código aqui
df['Ano'] = df['data_status'].str.split('/').str[-1]

In [25]:
# 4) Seu código aqui
df['Ano'].value_counts()

2012    11
2009     9
2013     7
2011     6
2002     4
2016     4
2014     4
2010     3
2006     2
2008     2
Name: Ano, dtype: int64

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [2]:
#1) Seu código aqui
def consulta_api(uf:str,tipo_interessado:str):
    url = f'http://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf={uf}&tipo_interessado={tipo_interessado}'
    r = requests.get(url)
    data_json = r.json()
    df = pd.DataFrame(data_json['items'])
    return df

In [23]:
# 2) Seu código aqui
dataframe_mg = consulta_api('MG','Estado')
dataframe_mg[dataframe_mg['status'] == 'Arquivado por decurso de prazo']['status'].value_counts()


Arquivado por decurso de prazo    1
Name: status, dtype: int64

In [22]:
# 3) Seu código aqui
dataframe_ba = consulta_api('BA','Município')
dataframe_ba[dataframe_ba['status'] == 'Deferido']['status'].value_counts()

Deferido    425
Name: status, dtype: int64

In [ ]:
# 4) Seu código aqui
estado_ba = consulta_api('BA','Estado')
estado_ba.to_csv('estado_ba.csv')